In [ ]:
import pandas as pd

In [ ]:
file = "data/database_list_2018_06_04.xlsx"

In [ ]:
df = pd.read_excel(file).fillna('False')

In [ ]:
plane_type_scheme, _ = SkosConceptScheme.objects.get_or_create(
    dc_title='plane_type'
)
reason_of_crash_scheme, _ = SkosConceptScheme.objects.get_or_create(
    dc_title='reason_of_crash'
)
function_in_plane_scheme, _ = SkosConceptScheme.objects.get_or_create(
    dc_title='function_in_plane'
)
rank_scheme, _ = SkosConceptScheme.objects.get_or_create(
    dc_title='rank'
)
destiny_stated_scheme, _ = SkosConceptScheme.objects.get_or_create(
    dc_title='destiny_stated'
)
destiny_checked_scheme, _ = SkosConceptScheme.objects.get_or_create(
    dc_title='destiny_checked'
)

In [ ]:
for i, row in df.iterrows():
    bomb, _ = Bomber.objects.get_or_create(
        macr_nr=row['MACR-Nr']
    )
    sqr_wrt_name = "{}__{}__{}".format(
        row['Air Force'], row['Bomber Group'], row['Squadron']
    )
    sqd, _ = Institution.objects.get_or_create(
        legacy_id=sqr_wrt_name
    )
    sqd.written_name = sqr_wrt_name
    sqd.save()
    bomb.squadron = sqd
    
    if row['Datum Absturz'] != 'False':
        bomb.date_of_crash = row['Datum Absturz']
    
    if row['Angriffsziel'] != 'False':
        target_place, _ = Place.objects.get_or_create(
            name=row['Angriffsziel']
        )
        bomb.target_place = target_place
    
    if row['Sichtung-Ort'] != 'False':
        last_seen, _ = Place.objects.get_or_create(
            name=row['Sichtung-Ort']
        )
        bomb.last_seen = last_seen

    if row['Absturzort'] != 'False':
        crash_place, _ = Place.objects.get_or_create(
            name=row['Absturzort']
        )
        bomb.crash_place = crash_place
    
    if row['Flugzeugtyp'] != 'False':
        plane_type, _ = SkosConcept.objects.get_or_create(
            pref_label=row['Flugzeugtyp']
        )
        plane_type.scheme.set([plane_type_scheme])
        bomb.plane_type = plane_type
    
    if row['Flugzeugname'] != 'False':
        bomb.name = row['Flugzeugname']
    
    if row['Absturzursache'] != 'False':
        reason_of_crash, _ = SkosConcept.objects.get_or_create(
            pref_label=row['Absturzursache']
        )
        reason_of_crash.scheme.set([reason_of_crash_scheme])
        bomb.reason_of_crash = reason_of_crash
    
    if row['Crewname (Pilot)'] != 'False':
        bomb.crew_name = row['Crewname (Pilot)']
        
    if row['Flugzeugnr.'] != 'False':
        bomb.plane_id = row['Flugzeugnr.']
        
    if row['Sichtung-Koord'] != 'False':
        bomb.sicht_koord = row['Sichtung-Koord']
        
    if row['Sichtung-Land'] != 'False':
        bomb.sicht_land = row['Sichtung-Land']
    
    if row['Sichtung-Ort'] != 'False':
        bomb.sicht_ort = row['Sichtung-Ort']
        
    if row['Anmerkung'] != 'False':
        bomb.anmerkung = row['Anmerkung']
        
    if row['Uhrzeit'] != 'False':
        bomb.uhrzeit = row['Uhrzeit']
        
    if row['Uhrzeit Absturz'] != 'False':
        bomb.uhrzeit_absturz = row['Uhrzeit Absturz']   
     
    bomb.save()

In [ ]:
for i, row in df.iterrows():
    pers, _ = Person.objects.get_or_create(
        dog_tag=row['Kennnummer']
    )
    
    if row['Familienname'] != 'False':
        pers.name = row['Familienname']
        
    if row['Name Kontrolle'] != 'False':
        pers.written_name = row['Name Kontrolle']    
    
    if row['Vorname'] != 'False':
        pers.forename = row['Vorname']
    
    if row['Mittelname'] != 'False':
        pers.middle_name=row['Mittelname']
    
    if row['Funktion'] != 'False':
        function_in_plane, _ = SkosConcept.objects.get_or_create(
            pref_label=row['Funktion']
        )
        function_in_plane.scheme.set([function_in_plane_scheme])
        pers.function_in_plane = function_in_plane
      
    if row['Dienstgrad'] != 'False':
        rank, _ = SkosConcept.objects.get_or_create(
            pref_label=row['Dienstgrad']
        )
        rank.scheme.set([rank_scheme])
        pers.rank = rank
        
    if row['Eintrag MACR'] != 'False':
        destiny_stated, _ = SkosConcept.objects.get_or_create(
            pref_label=row['Eintrag MACR']
        )
        destiny_stated.scheme.set([destiny_stated_scheme])
        pers.destiny_stated = destiny_stated
    
    if row['Details'] != 'False':
        pers.detail=row['Details']
        
    if row['Schicksal'] != 'False':
        destiny_checked, _ = SkosConcept.objects.get_or_create(
            pref_label=row['Schicksal']
        )
        destiny_checked.scheme.set([destiny_checked_scheme])
        pers.destiny_checked = destiny_checked
        
    bomb = Bomber.objects.get(
        macr_nr=row['MACR-Nr']
    )
    
    pers.part_of_bomber = bomb
              
    pers.save()

In [ ]:
for i, row in df.iterrows():
    try:
        pers = Person.objects.get(
            dog_tag=row['Kennnummer']
        )
    except ObjectDoesNotExist:
        pers = None
    
    if pers:
        if row['Stationen Gefangenschaft'] != 'False':
            for x in row['Stationen Gefangenschaft'].split('\n'):
                ps, _ = PrisonStation.objects.get_or_create(
                    legacy_id=x.strip()
                )
                ps.name = x.strip()
                ps.save()
                
                p_ps, _ = PersonPrison.objects.get_or_create(
                    related_person=pers,
                    related_prisonstation=ps
                )